In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import re

# 역 주소 전처리

In [2]:
df = pd.read_excel('역주소.xlsx')
df = df[['역 명', '역 주소']]

stat = df['역 명'].tolist()
add = df['역 주소'].tolist()

In [3]:
station=[]
for i in stat:
    j = re.sub(r"\(.*\)*", '', i)
    station.append(j)
    
df['역 명'] = station

In [4]:
add_list=[]
for i in add:
    j = i[0:3]
    add_list.append(j)
    
df['소속시'] = add_list

In [5]:
list_num=[]
list_num.append(df.index[df['소속시'] == '서울시'].tolist())

list_num[0]
    
df = df.loc[list_num[0]]

In [6]:
add = df['역 주소'].tolist()
gu_list=[]
for i in add:
    i2 = i.split()
    k=0
    for j in i2:
        k+=1
        if k==2:
            gu_list.append(j)

df['역 주소'] = gu_list

In [7]:
df = df[['역 명', '역 주소']]
df.columns = ['역명', '소재지']
df = df.drop_duplicates(['역명'], keep = 'first')
df = df.reset_index(drop=True)

# 1~8호선 승하차 전처리

In [8]:
data = pd.read_csv('subway/CARD_SUBWAY_MONTH_201804.csv', encoding='cp949')
data = data[['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수']]

In [9]:
line = ['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선']
line_num2=[]
for num in line:
    line_num2.append(data.index[data['노선명'] == num].tolist())
    
aaa=list(itertools.chain.from_iterable(line_num2))
data = data.loc[aaa]

stat = data['역명'].tolist()
station=[]
for i in stat:
    j = re.sub(r"\(.*\)", '', i)
    station.append(j)
    
data['역명'] = station

In [10]:
merge_outer = pd.merge(data, df, how='outer',on='역명')
merge_outer = merge_outer.dropna(axis=0)

# 편의시설 전처리

In [11]:
df22 = pd.read_excel('편의시설.xlsx', header=1)
df22 = df22.fillna(0)
df22 = df22.drop_duplicates(['역명'], keep = 'first')

In [12]:
merge_outer2 = pd.merge(merge_outer, df22, how='outer',on='역명')
merge_outer2 = merge_outer2.dropna(axis=0)
merge_outer2

,사용일자,노선명,역명,승차총승객수,하차총승객수,소재지,엘리베이터,에스컬레이터,휠체어리프트,이동식 안전발판,전동휠체어급속충전기,장애인 화장실,물품보관함,무인민원발급기,현급지급기,위생용품자판기,자동사진기,유아수유실
0,20180401.0,1호선,시청,11971.0,10936.0,중구,3.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0
1,20180402.0,1호선,시청,27374.0,27320.0,중구,3.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0
2,20180403.0,1호선,시청,29108.0,28864.0,중구,3.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0
3,20180404.0,1호선,시청,29023.0,28897.0,중구,3.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0
4,20180405.0,1호선,시청,29204.0,28545.0,중구,3.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658,20180426.0,8호선,복정,14693.0,11982.0,송파구,1.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
7659,20180427.0,8호선,복정,14985.0,12156.0,송파구,1.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
7660,20180428.0,8호선,복정,9955.0,8011.0,송파구,1.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
7661,20180429.0,8호선,복정,7558.0,6487.0,송파구,1.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


# 생활인구 전처리

In [14]:
data22 = pd.read_csv('생활인구/자치구단위 서울생활인구 일별 집계표_201804.csv', encoding='cp949')
data22 = data22[['기준일ID', '시군구명', '총생활인구수', '내국인생활인구수', '장기체류외국인인구수', '단기체류외국인인구수', '서울외유입인구수']]
data22.columns = ['사용일자', '소재지', '총생활인구수', '내국인생활인구수', '장기체류외국인인구수', '단기체류외국인인구수', '서울외유입인구수']

,사용일자,소재지,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,서울외유입인구수
0,20180430,서울시,1.164101e+07,1.101624e+07,412251.5015,212520.0885,1.609988e+06
1,20180430,종로구,3.957497e+05,3.542437e+05,20181.1523,21324.9369,9.086625e+04
2,20180430,중구,4.228838e+05,3.276555e+05,24855.5851,70372.6617,1.051082e+05
3,20180430,용산구,3.342199e+05,3.032816e+05,18705.8255,12232.4172,5.627041e+04
4,20180430,성동구,3.767996e+05,3.582770e+05,16640.5707,1881.9677,4.388366e+04
...,...,...,...,...,...,...,...
671,20180405,관악구,5.272641e+05,5.036422e+05,22109.7135,1512.2282,3.352409e+04
672,20180405,서초구,6.570409e+05,6.366557e+05,14811.8716,5573.3269,1.595478e+05
673,20180405,강남구,9.128639e+05,8.713715e+05,22101.3558,19390.9906,2.261431e+05
674,20180405,송파구,7.733757e+05,7.531781e+05,14761.5934,5436.0700,1.140617e+05


In [17]:
merge_outer3 = pd.merge(merge_outer2, data22, how='outer',on=['사용일자', '소재지'])
merge_outer3 = merge_outer3.dropna(axis=0)
merge_outer3 = merge_outer3.astype({'사용일자': str})

,사용일자,노선명,역명,승차총승객수,하차총승객수,소재지,엘리베이터,에스컬레이터,휠체어리프트,이동식 안전발판,...,무인민원발급기,현급지급기,위생용품자판기,자동사진기,유아수유실,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,서울외유입인구수
96,20180405.0,1호선,시청,29204.0,28545.0,중구,3.0,3.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082
97,20180405.0,2호선,시청,31653.0,29963.0,중구,3.0,3.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082
98,20180405.0,1호선,서울,59017.0,56738.0,중구,4.0,3.0,1.0,1.0,...,1.0,2.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082
99,20180405.0,4호선,서울,16431.0,25354.0,중구,4.0,3.0,1.0,1.0,...,1.0,2.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082
100,20180405.0,2호선,을지로입구,58177.0,58250.0,중구,4.0,6.0,0.0,1.0,...,1.0,3.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7598,20180426.0,7호선,가산디지털단지,49940.0,50995.0,금천구,4.0,12.0,4.0,1.0,...,0.0,1.0,1.0,0.0,1.0,252862.3912,236263.8886,15063.6664,1534.8362,65162.7899
7599,20180427.0,7호선,가산디지털단지,49908.0,50331.0,금천구,4.0,12.0,4.0,1.0,...,0.0,1.0,1.0,0.0,1.0,249771.8412,233205.6369,14998.7963,1567.4080,64520.5537
7600,20180428.0,7호선,가산디지털단지,21106.0,19164.0,금천구,4.0,12.0,4.0,1.0,...,0.0,1.0,1.0,0.0,1.0,222987.1298,205506.6114,15694.5555,1785.9629,37562.7902
7601,20180429.0,7호선,가산디지털단지,16076.0,14457.0,금천구,4.0,12.0,4.0,1.0,...,0.0,1.0,1.0,0.0,1.0,225903.4132,206841.7856,17343.5782,1718.0495,35014.5843


In [30]:
day = merge_outer3['사용일자'].tolist()
daycut=[]

for i in day:
    j = i[0:6]
    daycut.append(j)
    
merge_outer3['사용월'] = daycut
merge_outer3

,사용일자,노선명,역명,승차총승객수,하차총승객수,소재지,엘리베이터,에스컬레이터,휠체어리프트,이동식 안전발판,...,현급지급기,위생용품자판기,자동사진기,유아수유실,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,서울외유입인구수,사용월
96,20180405,1호선,시청,29204.0,28545.0,중구,3.0,3.0,0.0,1.0,...,1.0,1.0,0.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804
97,20180405,2호선,시청,31653.0,29963.0,중구,3.0,3.0,0.0,1.0,...,1.0,1.0,0.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804
98,20180405,1호선,서울,59017.0,56738.0,중구,4.0,3.0,1.0,1.0,...,2.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804
99,20180405,4호선,서울,16431.0,25354.0,중구,4.0,3.0,1.0,1.0,...,2.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804
100,20180405,2호선,을지로입구,58177.0,58250.0,중구,4.0,6.0,0.0,1.0,...,3.0,1.0,1.0,0.0,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7598,20180426,7호선,가산디지털단지,49940.0,50995.0,금천구,4.0,12.0,4.0,1.0,...,1.0,1.0,0.0,1.0,252862.3912,236263.8886,15063.6664,1534.8362,65162.7899,201804
7599,20180427,7호선,가산디지털단지,49908.0,50331.0,금천구,4.0,12.0,4.0,1.0,...,1.0,1.0,0.0,1.0,249771.8412,233205.6369,14998.7963,1567.4080,64520.5537,201804
7600,20180428,7호선,가산디지털단지,21106.0,19164.0,금천구,4.0,12.0,4.0,1.0,...,1.0,1.0,0.0,1.0,222987.1298,205506.6114,15694.5555,1785.9629,37562.7902,201804
7601,20180429,7호선,가산디지털단지,16076.0,14457.0,금천구,4.0,12.0,4.0,1.0,...,1.0,1.0,0.0,1.0,225903.4132,206841.7856,17343.5782,1718.0495,35014.5843,201804


# 유무임 비율 전처리

In [42]:
data333 = pd.read_csv('유무임/서울시 지하철 호선별 역별 유_무임 승하차 인원 정보_201804.csv', encoding='cp949')
data333 = data333[['사용월', '호선명', '지하철역', '유임승차인원', '무임승차인원', '유임하차인원', '무임하차인원']]
data333.columns = ['사용월', '노선명', '역명', '유임승차인원', '무임승차인원', '유임하차인원', '무임하차인원']
data333 = data333.astype({'사용월': str})

In [43]:
stat = data333['역명'].tolist()
station=[]
for i in stat:
    j = re.sub(r"\(.*\)", '', i)
    station.append(j)
    
data333['역명'] = station

In [44]:
merge_outer4 = pd.merge(merge_outer3, data333, how='outer',on=['사용월', '노선명', '역명'])
merge_outer4 = merge_outer4.dropna(axis=0)
merge_outer4

,사용일자,노선명,역명,승차총승객수,하차총승객수,소재지,엘리베이터,에스컬레이터,휠체어리프트,이동식 안전발판,...,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,서울외유입인구수,사용월,유임승차인원,무임승차인원,유임하차인원,무임하차인원
0,20180405,1호선,시청,29204.0,28545.0,중구,3.0,3.0,0.0,1.0,...,420670.1342,337041.4253,24779.2503,58849.4586,107256.9082,201804,644945,96564,637296,95277
1,20180406,1호선,시청,30722.0,30424.0,중구,3.0,3.0,0.0,1.0,...,425432.5912,335502.8792,25502.8055,64426.9066,107964.0992,201804,644945,96564,637296,95277
2,20180407,1호선,시청,19756.0,18920.0,중구,3.0,3.0,0.0,1.0,...,341995.2966,248401.4392,24052.8432,69541.0142,73546.3044,201804,644945,96564,637296,95277
3,20180408,1호선,시청,12676.0,11090.0,중구,3.0,3.0,0.0,1.0,...,297137.9225,211241.4105,22580.8032,63315.7088,52395.9731,201804,644945,96564,637296,95277
4,20180409,1호선,시청,27081.0,26888.0,중구,3.0,3.0,0.0,1.0,...,410391.3834,331299.9194,25049.6318,54041.8322,107845.0436,201804,644945,96564,637296,95277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,20180426,7호선,가산디지털단지,49940.0,50995.0,금천구,4.0,12.0,4.0,1.0,...,252862.3912,236263.8886,15063.6664,1534.8362,65162.7899,201804,1118426,84123,1121977,79710
6585,20180427,7호선,가산디지털단지,49908.0,50331.0,금천구,4.0,12.0,4.0,1.0,...,249771.8412,233205.6369,14998.7963,1567.4080,64520.5537,201804,1118426,84123,1121977,79710
6586,20180428,7호선,가산디지털단지,21106.0,19164.0,금천구,4.0,12.0,4.0,1.0,...,222987.1298,205506.6114,15694.5555,1785.9629,37562.7902,201804,1118426,84123,1121977,79710
6587,20180429,7호선,가산디지털단지,16076.0,14457.0,금천구,4.0,12.0,4.0,1.0,...,225903.4132,206841.7856,17343.5782,1718.0495,35014.5843,201804,1118426,84123,1121977,79710
